In [ ]:
!pip install transformers

In [ ]:
import re
import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
with open('esenin.txt', 'r') as f:
  text = f.read()

text = re.findall(r'\n\t\t.+\n\t\t', text)
text = [item.replace('\n\t\t', '') for item in text]
text[:10]

['Стихи мои,',
 '…И послушайте песню про горе,',
 'Думы печальные, думы глубокие,',
 'Думы от счастия вечно далекие,',
 'Белая береза',
 'Принакрылась снегом,',
 'Погуляйте, ратнички,',
 'Лошади запряжены,',
 'Повестили под окнами сотские',
 'Загыгыкали бабы слободские,']

In [ ]:
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write('\n'.join(text) + '\n')

In [ ]:
model_name = "ai-forever/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=32)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
training_args = TrainingArguments(
    output_dir="./finetuned",         # The output directory
    overwrite_output_dir=True,        # Overwrite the content of the output dir
    num_train_epochs=40,              # number of training epochs
    per_device_train_batch_size=20,   # batch size for training
    per_device_eval_batch_size=32,    # batch size for evaluation
    warmup_steps=9,                   # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=5
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5), None)
)

In [ ]:
trainer.train()

Step,Training Loss
500,3.184800


TrainOutput(global_step=520, training_loss=3.168393428509052, metrics={'train_runtime': 1172.494, 'train_samples_per_second': 45.339, 'train_steps_per_second': 0.443, 'total_flos': 2994014951571456.0, 'train_loss': 3.168393428509052, 'epoch': 38.81})

In [ ]:
model.save_pretrained('model_esenin')

In [ ]:
def generate(input_text):
  input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda:0')

  model.eval()
  with torch.no_grad():
      out = model.generate(
          input_ids,
          do_sample=True,
          num_beams=3,
          temperature=1.5,
          top_p=2.2,
          max_length=100
      )
  generated_text = list(map(tokenizer.decode, out))[0]
  print(generated_text)

In [ ]:
generate("Кто я? Что я? Только лишь мечтатель,")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Кто я? Что я? Только лишь мечтатель,
Я ищу счастья, покоя и уюта,
Но не здесь ли ты, моя муза,
Ты давно уж ушла от меня в дальнюю дорогу,
Как печально, печально! Я искал тебя, искал и не мог!
Ужели это ты, моя милая муза?
Ты ли это, моя милая муза?
Ужели это ты, моя милая муза?
Ужели это ты, моя милая му


In [ ]:
generate("Кто я?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Кто я?
Кто ты?
Я знаю —
Я помню —
Я все забыл.
Я хочу в туманы,
Только не здесь.
Дрогнул вечер.
И я вижу —
Небо плачет.
Плачет о чем-то дорогом
Слишком мало я жил.
Может быть, это снится
Я не знаю.
Не помню, что я видел.
Я не умею сказать.
Может быть, я сплю?
Может быть


In [ ]:
generate(" Друг мой, друг мой,")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Друг мой, друг мой,
Низкий дом без крыши,
Но и дома без стропил
Грустит ветер,
Видно, он помнит
Слышен плач детей,
Подымает усталый
Клен ты мой опавший,
Храни тебя бог.
Был бы жив Ленин,
Вынул б он эти строки
И сказал бы он:
«Слишком часто мы
Не в силах мы
Слишком часто
Оттого что


In [ ]:
generate("Зачем все это?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Зачем все это? Зачем?
Ведь все равно я любить не перестану,
Все равно буду я ночи и дни считать,
Ах, зачем же я так часто в думах хожу,
Ах, зачем же я по ночам, прижавшись к изголовью,
Буду думать о том, как бы мне под одною с тобою крышей
Как же я теперь счастлив, что не умею так любить!
Ах, зачем ж я теперь так часто по ночам, прижавшись к изголовью,

